In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
sc

In [ ]:
raw_graph = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/graph.tsv")

In [ ]:
raw_graph.collect()

In [ ]:
import re
import sys
from operator import add

In [ ]:
def computeContribs(urls, rank):
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [ ]:
def parseNeighbors(urls):
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [ ]:
links = raw_graph.map(parseNeighbors)\
                 .distinct()\
                 .groupByKey()

In [ ]:
links.collect()

In [ ]:
ranks = links.map(lambda url_neighbours: (url_neighbours[0], 1/4))

In [ ]:
ranks.collect()

In [ ]:
links.join(ranks).collect()

In [ ]:
contribs = links.join(ranks).flatMap(lambda u: computeContribs(u[1][0], u[1][1]))

In [ ]:
contribs.collect()

In [ ]:
contribs.reduceByKey(add).collect()

In [ ]:
raw_graph = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/graph.tsv")
links = raw_graph.map(parseNeighbors)\
                 .distinct()\
                 .groupByKey()
ranks = links.map(lambda url_neighbours: (url_neighbours[0], 1/4))
iterations = 5
for iteration in range(iterations):
    contribs = links.join(ranks).flatMap(lambda u: computeContribs(u[1][0], u[1][1]))
    ranks = contribs.reduceByKey(add)
    print("Iteration #{}".format(iteration))
    print("Current pagerank is:")
    print(sorted(ranks.collect()))

In [20]:
sc.stop()